In [5]:
import czifile as czi
import os
import json
import matplotlib.pyplot as plt
import numpy as np
import tifffile as tiff
from skimage.transform import downscale_local_mean
from collections import defaultdict
from tqdm.auto import tqdm

In [6]:
path = 'raw data'

### Extract Metadata info from czi filepath 

In [7]:
def get_czi_metadata(filepath, antibodies=None):
    czifile = czi.CziFile(filepath)
    metadata = czifile.metadata(raw=False)

    binning = int(metadata['ImageDocument']['Metadata']['Information']['Image']['Dimensions']['Channels']['Channel'][0]['DetectorSettings']['Binning'][0])
    channels = []
    for channel in metadata['ImageDocument']['Metadata']['Information']['Image']['Dimensions']['Channels']['Channel']:
        name = channel['Name']
        if name == 'AF647':
            name = 'Cy5'
        channels.append(name)
    bit_size = metadata['ImageDocument']['Metadata']['Information']['Image']['ComponentBitCount']
    size_x = metadata['ImageDocument']['Metadata']['Information']['Image']['SizeX']
    size_y = metadata['ImageDocument']['Metadata']['Information']['Image']['SizeY']
    pixel_size = float(metadata['ImageDocument']['Metadata']['ImageScaling']['ImagePixelSize'][:4])/binning/10
    
    info = {'filepath': filepath,
            'binning': binning,
            'sizeX': size_x,
            'sizeY': size_y,
            'pixel size': pixel_size,
            'bit size': bit_size,
            'channels': channels,
            'antibodies': antibodies}
    return info

### Save metadata in json file. Order of used antibodies can be defined manually in variable "antibodies"

In [ ]:
metadata = []
for root, dirs, files in os.walk(path):
    if 'NF' in root and 'small' not in root and 'überbelichtet' not in root and 'wrong binning' not in root:
        print(root)
                
        if 'ApoJ' in root and 'control\\5' in root:
            antibodies = ['NF', 'DAPI', 'ApoJ', 'Fabp7']
        elif 'ApoJ' in root and 'control\\6' in root:
            antibodies = ['NF', 'DAPI', 'Fabp7', 'ApoJ']
        elif 'ApoJ' in root:
            antibodies = ['NF', 'Fabp7', 'DAPI', 'ApoJ']
        elif 'DCX' in root and 'control\\6' in root:
            antibodies = [ 'DAPI', 'NF', 'DCX', 'Fabp7']
        elif 'DCX' in root:
            antibodies = ['NF', 'DCX', 'Fabp7', 'DAPI']
        elif 'Fabp7_GS' in root and 'control\\6' in root:
            antibodies = [ 'DAPI', 'NF', 'Fabp7', 'GS']
        elif 'Fabp7_GS' in root and 'control' in root:
            antibodies = ['NF', 'Fabp7', 'GS', 'DAPI']
        elif 'Fabp7_GS' in root and 'patients' in root:
            antibodies = ['DAPI', 'NF', 'Fabp7', 'GS']
        elif 'GFAP' in root and 'control\\6' in root:
            antibodies = [ 'DAPI', 'NF', 'GFAP', 'GS']
        elif 'GFAP' in root:
            antibodies = ['NF', 'GFAP', 'GS', 'DAPI']
        elif 'Iba1' in root and 'control\\6' in root:
            antibodies = ['DAPI', 'NF', 'Iba1', 'Map2']
        elif 'Iba1' in root and 'control' in root:
            antibodies = ['NF', 'Iba1', 'Map2', 'DAPI']
        elif 'Iba1' in root and 'patients\\6' in root:
            antibodies = ['NF', 'Iba1', 'Map2', 'DAPI']
        elif 'Iba1' in root and 'patients' in root:
            antibodies = ['DAPI', 'NF', 'Iba1', 'Map2']
        else:
            antibodies = None
        
        for file in files:
            filename = os.path.join(root, file)
            print(filename)
            img_metadata = get_czi_metadata(filename, antibodies)
            metadata.append(img_metadata)

In [32]:
with open('metadata.json', 'w') as file:
    file.write(json.dumps(metadata, indent=4))

### Class for converting czi files to tiff files (with function "convert_files). Images can be downscaled with a downscale factor. Next, channels can be saved seperately with the "split_channes" function.

In [9]:
class Datasource():
    def __init__(self, metadata_path):
        self.metadata_path = metadata_path
        self.channels_sorted = ['DAPI', 'AF488', 'Cy3', 'Cy5']
        with open(self.metadata_path) as file:
            self.metadata = json.load(file)
        self.number_of_files = len(self.metadata)
        #self.convert_files()
            
       
    def convert_files(self, export_path, downscale_factor=1):
        add = False       
        try:
            with open(self.metadata_path[:-5]+'_tifs.json') as file:
                self.metadata_tif = json.load(file)
            add = True
            print('add to data')
        except:
            print('new data')
        
        if add:
            tif_files_ori = []
            for tif_file in self.metadata_tif:
                tif_files_ori.append(tif_file['filepath_czi'])
                
            for n, file in enumerate(self.metadata):
                filepath = file['filepath']
                if filepath not in tif_files_ori:
                    self.metadata_tif.insert(n, file)
        else:      
            self.metadata_tif = self.metadata.copy()
        
        self.export_path = export_path
        self.downscale_factor = downscale_factor
        pathname_prev = ''
        for number, file in tqdm(enumerate(self.metadata)):
            # if image was converted to tiff, scip
            if 'tif' in self.metadata_tif[number]['filepath']:
                pass
            else:
                czi_img = czi.imread(file['filepath'])
                channels = file['channels']
                pixel_size = file['pixel size']
                antibodies = file['antibodies']
                antibodies_sorted = self.sort_antibodies(antibodies, channels)
                dirname = os.path.dirname(file['filepath'])
                pathname = os.path.join(self.export_path, dirname)
                if pathname != pathname_prev:
                    try:
                        os.makedirs(pathname)
                    except:
                        print(pathname+' already exists')
                    counter = 1
                else:
                    counter += 1
                filename = "{:04d}".format(counter)+'.tif'
                path_out = os.path.join(pathname, filename)
                self.czi2tif(czi_img, path_out, channels, pixel_size)
                self.update_metadata(number, path_out, file['filepath'], antibodies_sorted) 
                pathname_prev = pathname
        
        with open(self.metadata_path[:-5]+'_tifs.json', 'w') as file:
            file.write(json.dumps(self.metadata_tif, indent=4))
    
    def czi2tif(self, czi_img, path_out, channels, pixel_size):
        channel_len = len(channels)
        dict_img = {}
        for number, channel in zip(range(channel_len), channels):
            dict_img[channel] = czi_img[0,number,:,:,0]
        sorted_img = np.stack((dict_img[self.channels_sorted[0]], dict_img[self.channels_sorted[1]], dict_img[self.channels_sorted[2]], dict_img[self.channels_sorted[3]]))
        if self.downscale_factor == 1:
            image = sorted_img 
        else:
            image = downscale_local_mean(sorted_img, (1, self.downscale_factor, self.downscale_factor))
            image = np.uint16(image)
        ## to do: build in 12/14 bit to 16 bit scaling
        print('Saving: ', path_out)
        tiff.imwrite(path_out, image, imagej=True, 
                     resolution=(1./(pixel_size*self.downscale_factor), 1./(pixel_size*self.downscale_factor)), 
                     metadata={'unit': 'um', 'axes': 'CYX'})
    
    def sort_antibodies(self, antibodies, channels):
        antibodies = np.array(antibodies)
        channels = np.array(channels)
        channels_sorted = np.array(self.channels_sorted)
        indexes = []
        for channel in  channels_sorted:
            indexes.append(int(np.where(channels == channel)[0][0]))
        indexes = np.array(indexes)
        antibodies_sorted = antibodies[indexes]
        return antibodies_sorted.tolist()
    
    def update_metadata(self, number, path_out, path_in, antibodies_sorted):
        self.metadata_tif[number]['filepath'] = path_out
        self.metadata_tif[number]['filepath_czi'] = path_in
        self.metadata_tif[number]['binning'] = self.metadata[number]['binning']*self.downscale_factor
        self.metadata_tif[number]['pixel size'] = self.metadata[number]['pixel size']*self.downscale_factor
        self.metadata_tif[number]['channels'] = self.channels_sorted
        self.metadata_tif[number]['antibodies'] = antibodies_sorted
    
    def split_channels(self, staining='all'):
        
        with open(self.metadata_path[:-5]+'_tifs.json') as file:
            self.metadata_tif = json.load(file)  
        for file in tqdm(self.metadata_tif):
            if staining == 'all':
                self.save_channels(file)                        
            elif staining in file['filepath']:
                self.save_channels(file)
            else:
                pass
                                
    def save_channels(self, file):
        img = tiff.imread(file['filepath'])
        # get folder names from antibody list
        for n, antibody in enumerate(file['antibodies']):
            path = os.path.dirname(file['filepath'].replace("raw data","splitted"))+"\\"+antibody
            filename = os.path.basename(file['filepath'])
            filepath_new = os.path.join(path,filename)
            try:
                os.makedirs(path)
            except:
                pass
            image = img[n,:,:]
            print('Saving: ',filepath_new)
            tiff.imwrite(filepath_new, image)     

In [10]:
DRG_data = Datasource('metadata.json')

In [ ]:
DRG_data.convert_files(export_path='tif_data', downscale_factor=2)

In [ ]:
DRG_data.split_channels(staining='NF_Iba1_Map2')